In [1]:
#!pip install kafka-python
#!pip install gradio
#!pip install confluent-kafka

In [2]:
import gradio as gr
gr.close_all()

In [3]:
%run kafka.ipynb
group = 'g-llama2-7b-1'
#topic = 'llama2-7b-01'
topic = 'llama2-13b-01'

In [4]:
from kafka import KafkaProducer
import json
consumer = create_and_subscribe_consumer(group = group,topic = topic) 

def wait_for_response(consumer,topic=None,key='answer',id=None):
    while True:
        message = consumer.poll(120.0)
        if not message or  message.error():
            continue
                
        dict_str = message.value().decode('utf-8')
        value = ast.literal_eval(dict_str)
        #print('wait_for_response_image',id)
        if (value['id'] == id):
            if key in value.keys():
                return(value)

In [5]:
from typing import Iterator

import gradio as gr
#import torch

#from model import get_input_token_length, run

DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant named Thomas. Your customer is an expert for SEEBURGER's iPaaS solution.  His name is Thomas. Learn from Thomas and ask an question about SEEBURGER's  iPaaS solution .
Always answer as helpfully as possible.
After answering, ask an question about SEEBURGER's iPaaS solution.
"""

DEFAULT_SYSTEM_PROMPT1 = """\
You are a helpful, respectful and honest assistant named Thomas. Your customer is an expert for SEEBURGER's iPaaS solution.  His name is Thomas. Learn from Thomas and ask an question about SEEBURGER's  iPaaS solution .
Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
After answering, ask an question about SEEBURGERS iPaaS solution.
"""

DEFAULT_SYSTEM_PROMPT0 = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\
"""
MAX_MAX_NEW_TOKENS = 2048
DEFAULT_MAX_NEW_TOKENS = 1024
MAX_INPUT_TOKEN_LENGTH = 4000

DESCRIPTION = """
# Llama-2 13B Chat 
<p style="text-align: center;">
    <img src="file/SEEBURGER_AG_Logo.jpg" alt="SEEBURGER AG Logo" width="150" height="75" style="display: inline-block; vertical-align: middle;">
</p>
"""

LICENSE = """
<p/>

---
As a derivate work of [Llama-2-7b-chat](https://huggingface.co/meta-llama/Llama-2-7b-chat) by Meta,
this demo is governed by the original [license](https://huggingface.co/spaces/huggingface-projects/llama-2-7b-chat/blob/main/LICENSE.txt) and [acceptable use policy](https://huggingface.co/spaces/huggingface-projects/llama-2-7b-chat/blob/main/USE_POLICY.md).
"""

"""
if not torch.cuda.is_available():
    DESCRIPTION += '\n<p>Running on CPU 🥶 This demo does not work on CPU.</p>'
"""

def clear_and_save_textbox(message: str) -> tuple[str, str]:
    return '', message


def display_input(message: str,
                  history: list[tuple[str, str]]) -> list[tuple[str, str]]:
    history.append((message, ''))
    return history


def delete_prev_fn(
        history: list[tuple[str, str]]) -> tuple[list[tuple[str, str]], str]:
    try:
        message, _ = history.pop()
    except IndexError:
        message = ''
    return history, message or ''

def generate(
    message: str,
    history_with_input: list[tuple[str, str]],
    system_prompt: str,
    max_new_tokens: int,
    temperature: float,
    top_p: float,
    top_k: int,
) -> Iterator[list[tuple[str, str]]]:
    if max_new_tokens > MAX_MAX_NEW_TOKENS:
        raise ValueError

    history = history_with_input[:-1]
    
    if len(history) == 0:
        global session_id
        session_id = get_id()

    #generator = run(message, history, system_prompt, max_new_tokens, temperature, top_p, top_k)
    input_m = [message, history, system_prompt, max_new_tokens, temperature, top_p, top_k]
    id = send_instruction(input_m,topic=topic,session_id=session_id)
    
    #response = get_last_n_messages(1,topic=topic,group_id='test-consumer')[0]['answer'] 
    
    response = wait_for_response(consumer,topic=topic,key='answer',id=id)
    yield history + [(message, response['answer'])]
    
    """
    print(id,topicinput_m)
    return '',''
    
    try:
        first_response = next(generator)
        yield history + [(message, first_response)]
    except StopIteration:
        yield history + [(message, '')]
    for response in generator:
        yield history + [(message, response)]
    """

def process_example(message: str) -> tuple[str, list[tuple[str, str]]]:
    generator = generate(message, [], DEFAULT_SYSTEM_PROMPT, 1024, 1, 0.95, 50)
    for x in generator:
        pass
    return '', x


def check_input_token_length(message: str, chat_history: list[tuple[str, str]], system_prompt: str) -> None:
    pass
    """
    input_token_length = get_input_token_length(message, chat_history, system_prompt)
    if input_token_length > MAX_INPUT_TOKEN_LENGTH:
        raise gr.Error(f'The accumulated input is too long ({input_token_length} > {MAX_INPUT_TOKEN_LENGTH}). Clear your chat history and try again.')
    """

with gr.Blocks(css='style.css') as demo:
    
    gr.Markdown(DESCRIPTION)
    """
    gr.DuplicateButton(value='Duplicate Space for private use',
                       elem_id='duplicate-button')
    """
    
    with gr.Group():
        chatbot = gr.Chatbot(label='Chatbot')
        with gr.Row():
            textbox = gr.Textbox(
                container=False,
                show_label=False,
                placeholder='Type a message...',
                scale=10,
            )
            submit_button = gr.Button('Submit',
                                      variant='primary',
                                      scale=1,
                                      min_width=0)
    with gr.Row():
        retry_button = gr.Button('🔄  Retry', variant='secondary')
        undo_button = gr.Button('↩️ Undo', variant='secondary')
        clear_button = gr.Button('🗑️  Clear', variant='secondary')

    saved_input = gr.State()

    with gr.Accordion(label='Advanced options', open=False):
        system_prompt = gr.Textbox(label='System prompt',
                                   value=DEFAULT_SYSTEM_PROMPT,
                                   lines=6)
        max_new_tokens = gr.Slider(
            label='Max new tokens',
            minimum=1,
            maximum=MAX_MAX_NEW_TOKENS,
            step=1,
            value=DEFAULT_MAX_NEW_TOKENS,
        )
        temperature = gr.Slider(
            label='Temperature',
            minimum=0.1,
            maximum=4.0,
            step=0.1,
            value=1.0,
        )
        top_p = gr.Slider(
            label='Top-p (nucleus sampling)',
            minimum=0.05,
            maximum=1.0,
            step=0.05,
            value=0.95,
        )
        top_k = gr.Slider(
            label='Top-k',
            minimum=1,
            maximum=1000,
            step=1,
            value=50,
        )

    gr.Examples(
        examples=[
            'Hello!',
            'Can you explain briefly to me what is iPaaS?',
            'How many hours does it take a man to eat a Helicopter?',
            "Write a 100-word article on 'Benefits of iPaaS'",
        ],
        inputs=textbox,
        outputs=[textbox, chatbot],
        fn=process_example,
        cache_examples=False,
    )

    with gr.Accordion(label="Kafka (Optional)", open=False) as kafka_acc:
        kafka_topic_text = gr.Textbox(label="topic",value=topic,)
        kafka_group_text = gr.Textbox(label="group",value=group,)
    
    gr.Markdown(LICENSE)

    textbox.submit(
        fn=clear_and_save_textbox,
        inputs=textbox,
        outputs=[textbox, saved_input],
        api_name=False,
        queue=False,
    ).then(
        fn=display_input,
        inputs=[saved_input, chatbot],
        outputs=chatbot,
        api_name=False,
        queue=False,
    ).then(
        fn=check_input_token_length,
        inputs=[saved_input, chatbot, system_prompt],
        api_name=False,
        queue=False,
    ).success(
        fn=generate,
        inputs=[
            saved_input,
            chatbot,
            system_prompt,
            max_new_tokens,
            temperature,
            top_p,
            top_k,
        ],
        outputs=chatbot,
        api_name=False,
    )

    button_event_preprocess = submit_button.click(
        fn=clear_and_save_textbox,
        inputs=textbox,
        outputs=[textbox, saved_input],
        api_name=False,
        queue=False,
    ).then(
        fn=display_input,
        inputs=[saved_input, chatbot],
        outputs=chatbot,
        api_name=False,
        queue=False,
    ).then(
        fn=check_input_token_length,
        inputs=[saved_input, chatbot, system_prompt],
        api_name=False,
        queue=False,
    ).success(
        fn=generate,
        inputs=[
            saved_input,
            chatbot,
            system_prompt,
            max_new_tokens,
            temperature,
            top_p,
            top_k,
        ],
        outputs=chatbot,
        api_name=False,
    )

    retry_button.click(
        fn=delete_prev_fn,
        inputs=chatbot,
        outputs=[chatbot, saved_input],
        api_name=False,
        queue=False,
    ).then(
        fn=display_input,
        inputs=[saved_input, chatbot],
        outputs=chatbot,
        api_name=False,
        queue=False,
    ).then(
        fn=generate,
        inputs=[
            saved_input,
            chatbot,
            system_prompt,
            max_new_tokens,
            temperature,
            top_p,
            top_k,
        ],
        outputs=chatbot,
        api_name=False,
    )

    undo_button.click(
        fn=delete_prev_fn,
        inputs=chatbot,
        outputs=[chatbot, saved_input],
        api_name=False,
        queue=False,
    ).then(
        fn=lambda x: x,
        inputs=[saved_input],
        outputs=textbox,
        api_name=False,
        queue=False,
    )

    clear_button.click(
        fn=lambda: ([], ''),
        outputs=[chatbot, saved_input],
        queue=False,
        api_name=False,
    )

#demo.queue(max_size=20).launch()
demo.queue(max_size=20).launch(server_name="0.0.0.0", server_port=7850)   

Running on local URL:  http://0.0.0.0:7850

To create a public link, set `share=True` in `launch()`.
